# 5.5 Indexing expressions

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In algebraic notation, the use of sets is indicated informally by phrases such as "for
all $i \in P$ " or "`for t = 1, ..., T`" or "for all $j \in C$ such that $c_j > 0$." The AMPL counterpart
is the indexing expression that appears within braces `{ ... }` in nearly all of our examples.
An indexing expression is used whenever we specify the set over which a model
component is indexed, or the set over which a summation runs. Since an indexing
expression defines a set, it can be used in any place where a set is appropriate.

The simplest form of indexing expression is just a set name or expression within
braces. We have seen this in parameter declarations such as these from the multiperiod
production model of [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4):
```
param rate {PROD} > 0;
param avail {1..T} >= 0;
```
Later in the model, references to these parameters are subscripted with a single set member,
in expressions such as `avail[t]` and `rate[p]`. Variables can be declared and
used in exactly the same way, except that the keyword `var` takes the place of `param`.

The names such as `t` and `i` that appear in subscripts and other expressions in our
models are examples of *dummy indices* that have been defined by indexing expressions.
In fact, any indexing expression may optionally define a dummy index that runs over the
specified set. Dummy indices are convenient in specifying bounds on parameters:
```
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];
```
and on variables:
```
var Buy {j in FOOD} >= f_min[j], <= f_max[j];
```
They are also essential in specifying the sets over which constraints are defined, and the
sets over which summations are done. We have often seen these uses together, in declarations
such as
```
subject to Time {t in 1..T}:
  sum {p in PROD} (1/rate[p]) * Make[p,t] <= avail[t];
```
and
```
subject to Diet_Min {i in MINREQ}:
  sum {j in FOOD} amt[i,j] * Buy[j] >= n_min[i];
```
An indexing expression consists of an index name, the keyword `in`, and a set expression
as before. We have been using single letters for our index names, but this is not a
requirement; an index name can be any sequence of letters, digits, and underscores that is
not a valid number, just like the name for a model component.

Although a name defined by a model component's declaration is known throughout
all subsequent statements in the model, the definition of a dummy index name is effective
only within the *scope* of the defining indexing expression. Normally the scope is evident
from the context. For instance, in the `Diet_Min` declaration above, the scope of `{i in
MINREQ}` runs to the end of the statement, so that `i` can be used anywhere in the description
of the constraint. On the other hand, the scope of `{j in FOOD}` covers only the
summand `amt[i,j] * Buy[j]`. The scope of indexing expressions for sums and other
iterated operators is discussed further in [Chapter 7](../07/07.md).

Once an indexing expression's scope has ended, its dummy index becomes undefined.
Thus the same index name can be defined again and again in a model, and in fact it is
good practice to use relatively few different index names. A common convention is to
associate certain index names with certain sets, so that for example `i` always runs over
`NUTR` and `j` always runs over `FOOD`. This is merely a convention, however, not a restriction
imposed by AMPL. Indeed, when we modified the diet model so that there was a
subset `MINREQ` of `NUTR`, we used `i` to run over `MINREQ` as well as `NUTR`. The opposite
situation occurs, for example, if we want to specify a constraint that the amount of each
food `j` in the diet is at least some fraction `min_frac[j]` of the total food in the diet:
```
subject to Food_Ratio {j in FOOD}:
  Buy[j] >= min_frac[j] * sum {jj in FOOD} Buy[jj];
```
Since the scope of `j` in FOOD extends to the end of the declaration, a different index `jj`
is defined to run over the set `FOOD` in the summation within the constraint.

As a final option, the set in an indexing expression may be followed by a colon (`:`)
and a logical condition. The indexing expression then represents only the subset of members
that satisfy the condition. For example,
```
{j in FOOD: f_max[j] - f_min[j] < 1}
```
describes the set of all foods whose minimum and maximum amounts are nearly the
same, and
```
{i in NUTR: i in MAXREQ or n_min[i] > 0}
```
describes the set of nutrients that are either in `MAXREQ` or for which `n_min` is positive.
The use of operators such as `or` and `<` to form logical conditions will be fully explained
in [Chapter 7](../07/07.md).

By specifying a condition, an indexing expression defines a new set. You can use the
indexing expression to represent this set not only in indexed declarations and summations,
but anywhere else that a set expression may appear. For example, you could say
either of
```
set NUTREQ = {i in NUTR: i in MAXREQ or n_min[i] > 0};
set NUTREQ = MAXREQ union {i in MINREQ: n_min[i] > 0};
```
to define NUTREQ to represent our preceding example of a set expression, and you could
use either of
```
set BOTHREQ = {i in MINREQ: i in MAXREQ};
set BOTHREQ = MINREQ inter MAXREQ;
```
to define `BOTHREQ` to be the set of all nutrients that have both minimum and maximum
requirements. It's not unusual to find that there are several ways of describing some
complicated set, depending on how you combine set operations and indexing expression
conditions. Of course, some possibilities are easier to read than others, so it's worth taking
some trouble to find the most readable. In [Chapter 6](../06/06.md) we also discuss efficiency considerations
that sometimes make one alternative preferable to another in specifying compound
sets.

In addition to being valuable within the model, indexing expressions are useful in
`get_data` and `display` statements to summarize characteristics of the data or solution. The following
example is based on the model of [Figure 5-1](../05/5_3_set_operations.ipynb#fig-5-1) and the data of [Figure 5-2](../05/5_3_set_operations.ipynb#fig-5-2):

In [2]:
%%writefile dietu.mod

set MINREQ;                     # nutrients with minimum requirements
set MAXREQ;                     # nutrients with maximum requirements
set NUTR = MINREQ union MAXREQ; # nutrients
set FOOD;                       # foods
param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];
param n_min {MINREQ} >= 0;
param n_max {MAXREQ} >= 0;
param amt {NUTR,FOOD} >= 0;
var Buy {j in FOOD} >= f_min[j], <= f_max[j];
minimize Total_Cost: sum {j in FOOD} cost[j] * Buy[j];
subject to Diet_Min {i in MINREQ}:
 sum {j in FOOD} amt[i,j] * Buy[j] >= n_min[i];
subject to Diet_Max {i in MAXREQ}:
 sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

Overwriting dietu.mod


In [10]:
import pandas as pd

MINREQ = ['A', 'B1', 'B2', 'C', 'CAL']
MAXREQ = ['A', 'NA', 'CAL']

df_food = pd.DataFrame(
    [
        ['BEEF', 3.19, 2, 10],
        ['CHK', 2.59, 2, 10],
        ['FISH', 2.29, 2, 10],
        ['HAM', 2.89, 2, 10],
        ['MCH', 1.89, 2, 10],
        ['MTL', 1.99, 2, 10],
        ['SPG', 1.99, 2, 10],
        ['TUR', 2.49, 2, 10]
    ],
    columns=['FOOD', 'cost', 'f_min', 'f_max']
).set_index('FOOD')

# As opposed to Figure 5-2, specify n_min and n_max values with dictionaries
n_min_dict = {'A' : 700, 'C' : 700, 'B1' : 0, 'B2' : 0, 'CAL' : 16000}
n_max_dict = {'A' : 20000, 'NA' : 50000, 'CAL' : 24000}

df_amt = pd.DataFrame(
    [
        ['BEEF', 60, 20, 10, 15, 938, 295],
        ['CHK', 8, 0, 20, 20, 2180, 770],
        ['FISH', 8, 10, 15, 10, 945, 440],
        ['HAM', 40, 40, 35, 10, 278, 430],
        ['MCH', 15, 35, 15, 15, 1182, 315],
        ['MTL', 70, 30, 15, 15, 896, 400],
        ['SPG', 25, 50, 25, 15, 1329, 370],
        ['TUR', 60, 20, 15, 10, 1397, 450]
    ],
    columns=['FOOD', 'A', 'C', 'B1', 'B2', 'NA', 'CAL']
).set_index('FOOD').T

# Clear previous data
ampl.reset()

# Load dietu model
ampl.read('dietu.mod')

ampl.set['MINREQ'] = MINREQ
ampl.set['MAXREQ'] = MAXREQ

# Send data
ampl.param['n_min'] = n_min_dict
ampl.param['n_max'] = n_max_dict
ampl.set_data(df_food, 'FOOD')
ampl.param['amt'] = df_amt

# Solve problem
ampl.solve(solver='highs')

# Display some sets
ampl.display('{j in FOOD: Buy[j] > f_min[j]}')
ampl.display('{i in MINREQ: Diet_Min[i].slack = 0}')



HiGHS 1.11.0: optimal solution; objective 74.27382022
2 simplex iterations
0 barrier iterations
set {j in FOOD: Buy[j] > f_min[j]}  := CHK MTL SPG;

set {i in MINREQ: (Diet_Min[i].slack) == 0}  := C CAL;



In [12]:
# We can also retrieve these sets as python lists...
buy_more_than_fmin = ampl.get_data('{j in FOOD: Buy[j] > f_min[j]}').to_list()
print('Food strictly more than fmin: ', buy_more_than_fmin)
diet_min_slack_zero = ampl.get_data('{i in MINREQ: Diet_Min[i].slack = 0}').to_list()
print('Nutrients for which we get only n_min: ', diet_min_slack_zero)

Food strictly more than fmin:  ['CHK', 'MTL', 'SPG']
Nutrients for which we get only n_min:  ['C', 'CAL']


Amplpy commands such as `display` or `get_data` are allowed to refer to variables and constraints in the condition
phrase of an indexing expression, as illustrated by the last two cells above. Within a model, however, only sets, parameters and dummy indices may be mentioned in any indexing expression.

The set `BOTHREQ` above might well be empty, in the case where every nutrient has
either a minimum or a maximum requirement in the data, but not both. Indexing over an
empty set is not an error. When a model component is declared to be indexed over a set
that turns out to be empty, AMPL simply skips generating that component. A sum over
an empty set is zero, and other iterated operators over empty sets have the obvious interpretations
(see A.4 xTODO).